- 保监会 分类模型 2 训练 

# 基本设置

In [1]:
##load packages, needed
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import dict_dbutils
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from pandas.io import sql
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
if 0:
    class StatsFeatures(BaseEstimator, TransformerMixin):
    
        def __init__(self):
            self.neg = set()
            f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
            for content in f:
                self.neg.add(content)
            f.close()

        def fit(self, X, y=None):
            return self

        def getcnt(self,x):        
            return len(list(set(x)))

        def getnegcnt(self,x):
            negcnt = 0
            words = x.split()
            for w in words:
                if w in self.neg:
                    negcnt = negcnt+1
            return negcnt
    
        def transform(self, X):
            data = []
            for x in X:
                if len(x) == 0:
                    length  = 1
                else :
                    length = len(x)
                data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                             self.getnegcnt(x),self.getnegcnt(x)/length])            
            return data

In [2]:
class StatsFeatures_cor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()
        dict_org=dict_dbutils.get_dicts()
        self.org=set(dict_org.keys())
        
        

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    def getorgcnttf(self,x):
        orgcnt=0
        orgtf=0
        words = x.split()
        words_set=set(words)
        for w in words_set:
            if w in self.org:
                orgcnt = orgcnt+1
                orgtf=orgtf+words.count(w)
        if(orgcnt>0):
            return orgcnt,orgtf
        else:
            return orgcnt,orgtf
    
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            orgcnt,orgtf=self.getorgcnttf(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length,
                         orgcnt,orgtf])            
        return data

In [ ]:
class StatsFeatures_01(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()
        dict_org=dict_dbutils.get_dicts()
        self.org=set(dict_org.keys())
        
        

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    def getorgcnttf(self,x):
        orgcnt=0
        orgtf=0
        words = x.split()
        words_set=set(words)
        for w in words_set:
            if w in self.org:
                orgcnt = orgcnt+1
                orgtf=orgtf+words.count(w)
        if(orgcnt>0):
            return orgcnt,orgtf
        else:
            return orgcnt,orgtf
    
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            orgcnt,orgtf=self.getorgcnttf(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length,
                         orgcnt,orgtf])            
        return data

## 上一版模型读入

In [3]:
# 上一版模型
# from sklearn.externals import joblib
# pipeline_old = joblib.load( "model/circ_cor_0625.pkl.z") 

# 导入数据

In [3]:
title_content = []
filename = 'data/result/all_titles+contents_pre.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    title_content.append(f.strip())
fid.close()
print(len(title_content))

7508


In [4]:
content = []
filename = 'data/result/all_contents_pre.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    content.append(f.strip())
fid.close()
print(len(title_content))

7508


In [5]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}

In [6]:
label = []
filename = 'data/result/all_labels_pre.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    label.append(f.strip())
fid.close()
label=[label_dic[x.strip()] for x in label]
print(len(label))

7508


In [7]:
title = []
filename = 'data/result/all_titles_pre.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    title.append(f.strip())
fid.close()
print(len(title))

7508


# 训练模型

## 分割训练集、测试集

In [8]:
X_train, X_test, y_train, y_test = train_test_split(title_content, label, test_size=0.1, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 6757
训练集-各类数量： Counter({8: 3054, 1: 1338, 7: 611, 5: 522, 4: 420, 3: 288, 2: 273, 6: 251})
测试集： 751
测试集-各类数量： Counter({8: 323, 1: 150, 7: 67, 5: 62, 2: 44, 4: 43, 3: 32, 6: 30})


## xgboost

### train

In [9]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

In [10]:
print(pipeline.score(X_train, y_train))

0.9979280745893148


### test

In [11]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8415446071904128
             precision    recall  f1-score   support

          1       0.83      0.87      0.85       150
          2       0.79      0.50      0.61        44
          3       0.81      0.53      0.64        32
          4       0.86      0.72      0.78        43
          5       0.75      0.74      0.75        62
          6       0.96      0.80      0.87        30
          7       0.85      0.90      0.87        67
          8       0.86      0.93      0.89       323

avg / total       0.84      0.84      0.84       751

confusion_matrix: 
[[131   2   0   0   4   0   1  12]
 [  6  22   1   1   2   1   1  10]
 [  1   0  17   0   1   0   3  10]
 [  1   1   0  31   1   0   0   9]
 [  7   1   1   0  46   0   4   3]
 [  1   0   0   0   2  24   0   3]
 [  0   1   1   0   1   0  60   4]
 [ 10   1   1   4   4   0   2 301]]


In [54]:
# 上一版模型 
y_pred_class = pipeline_old.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8652058432934927
             precision    recall  f1-score   support

          0       0.88      0.89      0.88       862
          1       0.85      0.84      0.84       644

avg / total       0.87      0.87      0.87      1506

confusion_matrix: 
[[764  98]
 [105 539]]


# 模型保存

In [12]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/circ_8classifier_0731.pkl.z")

['model/circ_8classifier_0731.pkl.z']

# 保存本文件

In [56]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)